In [1]:
%matplotlib inline
from matplotlib.pyplot import figure, show
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.models import Sequential

C:\Users\Ashish\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path = './Data/'

tf.logging.set_verbosity(tf.logging.ERROR)

filename = os.path.join(path,  'mpg.csv')
df = pd.read_csv(filename, na_values=['NA','?'])

In [3]:
df.drop('name',1,inplace=True)

In [4]:
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)

def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)




In [5]:
missing_median(df, 'horsepower')
x,y = to_xy(df, 'mpg')

C:\Users\Ashish\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [6]:
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=45)

In [7]:
model = Sequential()
model.add(Dense(50,
               input_dim=x.shape[1],
               activation='relu'))
model.add(Dense(25,
               activation='relu'))
model.add(Dense(10,
               kernel_regularizer=regularizers.l2(0.01),
               activity_regularizer=regularizers.l1(0.01),
               activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error',
            optimizer='adam')
monitor = EarlyStopping(monitor='val_loss',
                       min_delta=1e-3,
                       patience=5,
                       verbose=1,
                       mode='auto')
model.fit(x_train, y_train, validation_data=(x_test, y_test),
         callbacks=[monitor],
         verbose=0,
         epochs=1000)
pred = model.predict(x_test)

Epoch 00104: early stopping


In [9]:
# RMSE
score = np.sqrt(metrics.mean_squared_error(pred, y_test))
print(score)

4.1377573
